# 1. Install dependencies and download the data should take about 5 miutes

In [1]:
!pip install -U datasets google-cloud-storage dataquality  &> /dev/null
import os
from tqdm import tqdm
from datasets import load_dataset
import torch 
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from matplotlib import pyplot as plt
from google.cloud import storage

# download images
import os
os.system('gsutil cp -r \
  "gs://galileo-public-data/CV_datasets/Segmentation_Data" \
  . &> /dev/null')

dataset_path = "./Segmentation_Data"

IMG_SIZE = 256
NC = 21  # Number of classes

Copying gs://galileo-public-data/CV_datasets/Segmentation_Data/train/images/image_0.png...
/ [1 files][446.3 KiB/446.3 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://galileo-public-data/CV_datasets/Segmentation_Data/train/images/image_1.png...
Copying gs://galileo-public-data/CV_datasets/Segmentation_Data/train/images/image_2.png...
Copying gs://galileo-public-data/CV_datasets/Segmentation_Data/train/images/image_3.png...
Copying gs://galileo-public-data/CV_datasets/Segmentation_Data/train/images/image_4.png...
Copying gs://galileo-public-data/CV_datasets/Segmentation_Data/train/images/image_5.png...
Copying gs://galileo-public-data/CV_datasets/Segmentation_Data/train/images/image_6.png...
Copying gs://galileo-public-da

# 2. Create the dataset class


In [2]:
from typing import Optional, Dict, Union
from PIL import Image

class coco_hf_dataset_disk(torch.utils.data.Dataset):
    def __init__(self, 
                 dataset_path: str,
                 relative_img_path: str, 
                 relative_mask_path: str,
                 mask_transform: transforms=None, 
                 img_transform: transforms=None, 
                 size: int=1024,) -> None:
        """"
        COCO val dataset from galileo-public-data/CV_datasets/COCO_seg_val_5000/all_images
        downloaded and located on disk.
        If no paths are provided we download the dataset from GCS and save it to disk.

        :param dataset_path: path to dataset
        :param relative_img_path: path to images relative to the dataset path
        :param relative_maks_path: path to masks relative to the dataset path
        :param mask_transform: transforms to apply to masks
        :param img_transform: transforms to apply to images
        :param size: size of image and mask
        """
        super(coco_hf_dataset_disk, self).__init__()

        self.dataset_path = dataset_path
        self.relative_img_path = relative_img_path
        self.relative_mask_path = relative_mask_path
        self.images = sorted(os.listdir(os.path.join(dataset_path, relative_img_path)))
        self.masks = sorted(os.listdir(os.path.join(dataset_path, relative_mask_path)))
        # remove .DS_Store
        if self.images[0] == '.DS_Store':
            self.images = self.images[1:]
        if self.masks[0] == '.DS_Store':
            self.masks = self.masks[1:]

        num_images = len(self.images)
        num_masks = len(self.masks)
        print(f"Found dataset, there are {num_images} images and {num_masks} masks")

        self.mask_transform = mask_transform
        self.img_transform = img_transform

        self.class_dict = { 'background': 0, 'airplane': 1, 'bicycle': 2,
                            'bird': 3, 'boat': 4, 'bottle': 5, 'bus': 6,
                            'car': 7, 'cat': 8, 'chair': 9, 'cow': 10,
                            'dining table': 11,'dog': 12,'horse': 13,
                            'motorcycle': 14,'person': 15,'potted plant': 16,
                            'sheep': 17, 'couch': 18, 'train': 19, 'tv': 20}
        self.labels = [i for i in self.class_dict]
                        
        self.int2str = {v: k for k, v in self.class_dict.items()}
        self.size = size

    def __len__(self) -> int:
        # returns length of the dataset
        return len(self.images)

    def __getitem__(self, idx: int) -> Dict[str, Union[torch.Tensor, int, np.ndarray]]:
        # gets a single item from our dataset
        
        image_path = os.path.join(self.dataset_path, self.relative_img_path, self.images[idx])
        mask_path = os.path.join(self.dataset_path, self.relative_mask_path, self.masks[idx])
        image = Image.open(image_path)
        mask = Image.open(mask_path)

        # resize image and mask to given size
        unnormalized_image = image.copy().resize((self.size, self.size), resample=Image.NEAREST)
        unnormalized_image = transforms.ToTensor()(unnormalized_image)
        unnormalized_image = expand_gray_channel()(unnormalized_image)
        unnormalized_image = np.array(unnormalized_image)
        

        if self.img_transform:
            image = self.img_transform(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)
        
        return {'image': image,
                'image_path': image_path,
                'mask_path': mask_path,
                'mask': mask,
                'idx': idx,
                'unnormalized_image': unnormalized_image}


class expand_gray_channel:
    def __call__(self, tensor: torch.Tensor) -> torch.Tensor:
        # torch transform to expand gray channel to 3 channels
        if tensor.shape[0] > 3:
            tensor = tensor.unsqueeze(0)
        if tensor.shape[0] == 1:
            return tensor.expand(3, -1, -1)
        return tensor
    

# 3. Create the datasets, dataloaders, model and optimizer

In [3]:
img_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((IMG_SIZE, IMG_SIZE), interpolation=transforms.InterpolationMode.BICUBIC),
    expand_gray_channel(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
mask_transforms = transforms.Compose([
    transforms.PILToTensor(),
    transforms.Resize((IMG_SIZE, IMG_SIZE), interpolation=transforms.InterpolationMode.NEAREST),
])

train_image_path = 'train/images'
train_mask_path = 'train/masks'
validation_image_path = 'validation/images'
validation_mask_path = 'validation/masks'

train_dataset = coco_hf_dataset_disk(dataset_path=dataset_path,
                                    relative_img_path=train_image_path, 
                                    relative_mask_path=train_mask_path,
                                    img_transform=img_transforms,
                                     mask_transform=mask_transforms,
                                    size=IMG_SIZE)
validation_dataset = coco_hf_dataset_disk(dataset_path=dataset_path,
                                    relative_img_path=validation_image_path, 
                                    relative_mask_path=validation_mask_path,
                                    img_transform=img_transforms,
                                     mask_transform=mask_transforms,
                                    size=IMG_SIZE)
labels = train_dataset.labels
train_dataloader = DataLoader(train_dataset, batch_size = 6, shuffle=True, num_workers=4)
validation_dataloader = DataLoader(validation_dataset, batch_size = 6, shuffle=True, num_workers=4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = .00001)

Found dataset, there are 10 images and 10 masks
Found dataset, there are 6 images and 6 masks


Using cache found in /Users/derek/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/derek/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/derek/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# 4. Train with Dataquality  🔭

In [4]:
import dataquality as dq
from dataquality.integrations.cv.torch.semantic_segmentation import watch

# set to avoid being prompted for credentials
# %env GALILEO_CONSOLE_URL = "https://console.cloud.rungalileo.io/" 
# %env GALILEO_USERNAME = 
# %env GALILEO_PASSWORD = 

# dq.configure()
dq.init("semantic_segmentation", "Segmentation_Project", 'COCO_dataset')

watch(
    model,
    bucket_url='https://storage.googleapis.com/galileo-public-data/CV_datasets/COCO_segmentation_example_data',
    dataset_path=dataset_path,
    dataloaders={"training": train_dataloader,
                 "validation": validation_dataloader},
)
dq.set_labels_for_run(labels)


Welcome to Galileo Cloud v0.8.45!


2023-05-31 13:55:35.245593: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/derek/opt/anaconda3/lib/python3.9/site-packages/dataquality/core/__init__.py:27: GalileoWarning: configure is deprecated, use dq.set_console_url and dq.login
  warnings.warn(


Welcome to Galileo v0.8.45!
To skip this prompt in the future, set the following environment variable: GALILEO_CONSOLE_URL
📡 https://console.cloud.rungalileo.io
🔭 Logging you into Galileo

Go to https://console.cloud.rungalileo.io/get-token to generate a new API Key
🚀 You're logged in to Galileo as derek@rungalileo.io!
✨ Initializing new public project 'Segmentation_Project'
🏃‍♂️ Creating new run 'COCO_dataset'
🛰 Connected to new project 'Segmentation_Project', and new run 'COCO_dataset'.
We assume the dataloaders passed only have transforms that Tensor, Resize,         and Normalize the image and mask
‼ Any cropping or shearing transforms passed will lead to unexpected         results
See docs at https://dq.readthedocs.io/en/latest/ (placeholder) for more info         
 

Found layer "classifier" in model layers: "backbone, classifier"


/var/folders/hm/ll4g2nzj0jnckzf_w02pc2lm0000gn/T/ipykernel_5397/1713176154.py:68: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  unnormalized_image = image.copy().resize((self.size, self.size), resample=Image.NEAREST)


In [5]:
# train for one epoch

scaler = torch.cuda.amp.GradScaler()
epochs = 1

with torch.autocast('cuda'):
    for epoch in range(epochs):
        dq.set_epoch_and_split(epoch, "training")
        for j, sample in enumerate(tqdm(train_dataloader)):
            imgs, masks = sample['image'], sample['mask']
            out = model(imgs.to(device))

            # reshape to have loss for each pixel (bs * h * w, 21)\n",
            pred = out['out'].permute(0, 2, 3, 1).contiguous().view( -1, 21)
            masks = masks.long()
            msks_for_loss = masks.view(-1).to(device)

            loss = criterion(pred, msks_for_loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

/Users/derek/opt/anaconda3/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/Users/derek/opt/anaconda3/lib/python3.9/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/hm/ll4g2nzj0jnckzf_w02pc2lm0000gn/T/ipykernel_5397/1713176154.py:68: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  unnormalized_image = image.copy().resize((self.size, self.size), resample=Image.NEAREST)
100%|██████████| 2/2 [00:12<00:00,  6.29s/it]


In [6]:
dq.finish()

/var/folders/hm/ll4g2nzj0jnckzf_w02pc2lm0000gn/T/ipykernel_5397/1713176154.py:68: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  unnormalized_image = image.copy().resize((self.size, self.size), resample=Image.NEAREST)
/Users/derek/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/Users/derek/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


Logging 4 samples [########################################] 100.00% elapsed time  :     0.29s =  0.0m =  0.0h
Logging 6 samples [########################################] 100.00% elapsed time  :     0.18s =  0.0m =  0.0h
Logging 6 samples [########################################] 100.00% elapsed time  :     0.18s =  0.0m =  0.0h
 ☁️ Uploading Data
CuML libraries not found, running standard process. For faster Galileo processing, consider installing
`pip install 'dataquality[cuda]' --extra-index-url=https://pypi.nvidia.com/`


Uploading data to Galileo:   0%|          | 0.00/32.3k [00:00<?, ?B/s]

Processing data for upload:   0%|          | 0/2 [00:00<?, ?it/s]

Uploading data to Galileo:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Processing data for upload:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading data to Galileo:   0%|          | 0.00/34.7k [00:00<?, ?B/s]

[05/31/23 13:57:04] WARNING  could not close memmap for column                                   ]8;id=335036;file:///Users/derek/opt/anaconda3/lib/python3.9/site-packages/vaex/dataset_mmap.py\dataset_mmap.py]8;;\:]8;id=461726;file:///Users/derek/opt/anaconda3/lib/python3.9/site-packages/vaex/dataset_mmap.py#94\94]8;;\
                             /Users/derek/.galileo/logs/f0de2cdc-5940-4997-ab3c-c354e0120823/a93                   
                             acda7-8964-4d34-82fa-bdac4d91c372/input_data/validation/data_0.hdf5                   

Job default successfully submitted. Results will be available soon at https://console.cloud.rungalileo.io/insights?projectId=f0de2cdc-5940-4997-ab3c-c354e0120823&runId=a93acda7-8964-4d34-82fa-bdac4d91c372&split=training&metric=f1&depHigh=1&depLow=0&taskType=6
Waiting for job (you can safely close this window)...
	[training] 👀 Looking for data anomalies
	[validation] 👀 Looking for data anomalies
Done! Job finished with status completed
Click here to see your run! https://console.cloud.rungalileo.io/insights?projectId=f0de2cdc-5940-4997-ab3c-c354e0120823&runId=a93acda7-8964-4d34-82fa-bdac4d91c372&split=training&metric=f1&depHigh=1&depLow=0&taskType=6
🧹 Cleaning up
🧹 Cleaning up


'https://console.cloud.rungalileo.io/insights?projectId=f0de2cdc-5940-4997-ab3c-c354e0120823&runId=a93acda7-8964-4d34-82fa-bdac4d91c372&split=training&metric=f1&depHigh=1&depLow=0&taskType=6'